In [1]:
from bokeh.io import output_notebook, show
output_notebook()

Loading BokehJS ...

In [8]:
from bokeh.layouts import column, row
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Select, TextInput, Button, Paragraph
import numpy as np
import cv2
import os

os.environ["BOKEH_ALLOW_WS_ORIGIN"] = 'localhost:8889'

In [9]:
def modify_doc(doc):
    
    # create some widgets
    button = Button(label="Say HI")
    input = TextInput(value="Bokeh")
    output = Paragraph()

    # add a callback to a widget
    def update():
        output.text = "Hello, " + input.value
    button.on_click(update)

    # create a layout for everything
    layout = column(button, input, output)

    # add the layout to curdoc
    doc.add_root(layout)
    
# In the notebook, just pass the function that defines the app to show
# You may need to supply notebook_url, e.g notebook_url="http://localhost:8889" 


In [10]:
show(modify_doc) 

In [11]:
def get_data(N):
    return dict(x=np.random.normal(size=N), y=np.random.normal(size=N), r=np.random.normal(size=N) * 0.03)

COLORS = ["black", "firebrick", "navy", "olive", "goldenrod"]

def modify_doc(doc):
    source = ColumnDataSource(data=get_data(200))

    p = figure(tools="", toolbar_location=None)
    r = p.circle(x='x', y='y', radius='r', source=source,
                 color="navy", alpha=0.6, line_color="white")

    
    select = Select(title="Color", value="navy", options=COLORS)
    input = TextInput(title="Number of points", value="200")

    def update_color(attrname, old, new):
        r.glyph.fill_color = select.value
    select.on_change('value', update_color)

    def update_points(attrname, old, new):
        N = int(input.value)
        source.data = get_data(N)
    input.on_change('value', update_points)

    layout = column(row(select, input, width=400), row(p))

    doc.add_root(layout)

In [12]:
show(modify_doc) 

In [13]:
from bokeh.models import ColumnDataSource
import math

In [16]:

def modify_doc2(doc):
    p = figure(match_aspect=True)
    p.circle(x=0, y=0, radius=1, fill_color=None, line_width=2)
    
    # this is just to help the auto-datarange
    p.rect(0, 0, 2, 2, alpha=0)

    # this is the data source we will stream to
    source = ColumnDataSource(data=dict(x=[1], y=[0]))
    p.circle(x='x', y='y', size=12, fill_color='white', source=source)

    def update():
        x, y = source.data['x'][-1], source.data['y'][-1]

        # construct the new values for all columns, and pass to stream
        new_data = dict(x=[x*math.cos(0.1) - y*math.sin(0.1)], y=[x*math.sin(0.1) + y*math.cos(0.1)])
        source.stream(new_data, rollover=8)

    doc.add_periodic_callback(update, 150)
    doc.add_root(p)
    

In [15]:
show(modify_doc2)

-0.9352099151945389

1200

In [36]:
import math
import numpy as np
import pandas as pd

from bokeh.io import output_file, show
from bokeh.models import GraphRenderer, Oval, StaticLayoutProvider
from bokeh.palettes import Spectral8
from bokeh.plotting import figure

#from bokeh.io import output_notebook, show
#output_notebook()


In [37]:
#should be already defined///////
def pathToName(path = "out.mp4"):
    x = path.split("/")
    return x[-1].split(".")[0]
videoFile="./videos/mov3.mp4"
videoOut="./videos/"+pathToName(videoFile)+"out.mp4"
csvOut="./videos/"+pathToName(videoFile)+".csv"


In [28]:
def genBokeh_Skeleton():
    mySkeleton=pd.read_csv("mySkel.csv",sep=";",index_col=0)  
    mySkeleton["coord2d"]=mySkeleton["coord2d"].apply(cleanStrCoords)
    # graphbuilder- connections
    jSrc=[]
    jDst=[]
    jSrc.append(0)
    jDst.append(1)#head-neck
    jSrc.append(2)
    jDst.append(3)#right_shoulder - right_elbow
    jSrc.append(3)
    jDst.append(4)#right_elbow - right_wrist
    jSrc.append(5)
    jDst.append(6)#left_shoulder - left_elbow
    jSrc.append(6)
    jDst.append(7)#left_elbow - left_wrist
    jSrc.append(8)
    jDst.append(9)#right_hip - right_knee
    jSrc.append(9)
    jDst.append(10)#right_knee - right_foot
    jSrc.append(11)
    jDst.append(12)#left_hip - left_knee
    jSrc.append(12)
    jDst.append(13)#left_knee - left_foot
    jSrc.append(14)
    jDst.append(15)#right_eye - left_eye
    #jSrc.append(14)
    #jDst.append(1)#right_eye - head
    #jSrc.append(15)
    #jDst.append(1)#left_eye - head
    jSrc.append(5)
    jDst.append(2)#left_shoulder - right_shoulder
    jDst.append(2)
    jSrc.append(19)#right_shoulder - pelvis
    jDst.append(5)
    jSrc.append(19)#left_shoulder - pelvis
    jDst.append(8)
    jSrc.append(11)#right_hip - left_hip
    conns={'start':jSrc, 'end':jDst}
    return mySkeleton, conns

def cleanStrCoords(coords):
    if isinstance(coords, str):
        res=coords[1:-1].split(",")
        coords=(int(res[0]) , int(res[1]))
    return coords

In [29]:
def centerViewCoords(h, w):
    if h>w:
        xmin, ymin, xmax, ymax = 0, int((h-w)/2), w, w+int((h-w)/2)
    else:
        xmin, ymin, xmax, ymax = int((w-h)/2), 0, h+int((w-h)/2), h
    return xmin, ymin, xmax, ymax

In [34]:

def modify_doc(doc):

    frameNumber=50
    videoFile="./videos/mov3out.mp4"
    
    capture = cv2.VideoCapture(videoFile, cv2.CAP_FFMPEG)
    if (capture.isOpened() == False): 
        print("Unable to open video file")
    capture.set(cv2.CAP_PROP_POS_FRAMES, frameNumber)
    ret, frame = capture.read()
    if not ret:
        print("Can't receive frame (video end?). Exiting ...")
        frame=cv2.imread("./output_net.png")#error image
    capture.release()

    frameRGBA = cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA)
    frameRGBA = cv2.flip(frameRGBA, 0)
    h, w, c = frameRGBA.shape
    xmin, ymin, xmax, ymax = centeredViewCoords(h, w)

    p = figure(x_range=(xmin, xmax), y_range=(ymin, ymax), tooltips=[("x coord", "$x"), ("y", "$y"), ("value", "@image")])
    p.image_rgba(image=[frameRGBA], x=0, y=0, dw=w, dh=h)    
    layout = column(p)

    doc.add_root(layout)
    

In [33]:
show(modify_doc)

In [30]:
h=720
w=1280

xmin, ymin, xmax, ymax = centerViewCoords(h,w)
plot = figure(x_range=(xmin, xmax), y_range=(ymax, ymin),
              tools='', toolbar_location=None)
graph = GraphRenderer()
mySkeleton, conns=genBokeh_Skeleton()
graph.node_renderer.data_source.add(mySkeleton.index, 'index')
graph.node_renderer.data_source.add(mySkeleton['color'], 'color')
graph.node_renderer.glyph = Oval(height=12, width=12, fill_color='color')

graph.edge_renderer.data_source.data = conns
graph_layout = dict(zip(mySkeleton.index,  mySkeleton["coord2d"]))
graph.layout_provider = StaticLayoutProvider(graph_layout=graph_layout)
plot.renderers.append(graph)

In [31]:
show(plot)

In [24]:


mySkeleton, conns=genBokeh_Skeleton()

dict(zip(mySkeleton.index,  mySkeleton["coord2d"]))

{0: (639, 123),
 1: (617, 191),
 2: (572, 191),
 3: (527, 236),
 4: (527, 191),
 5: (662, 191),
 6: (729, 258),
 7: (729, 191),
 8: (595, 348),
 9: (595, 461),
 10: (595, 573),
 11: (662, 348),
 12: (640, 461),
 13: (640, 573),
 14: (617, 123),
 15: (639, 123),
 16: (595, 124),
 17: (640, 145),
 19: (628, 348)}

In [11]:
graph.edge_renderer.data_source.data

{'start': [0, 0, 0, 0, 0, 0, 0, 0], 'end': [0, 1, 2, 3, 4, 5, 6, 7]}

In [18]:
graph_layout

{0: (1.0, 0.0),
 1: (0.7071067811865476, 0.7071067811865475),
 2: (6.123233995736766e-17, 1.0),
 3: (-0.7071067811865475, 0.7071067811865476),
 4: (-1.0, 1.2246467991473532e-16),
 5: (-0.7071067811865477, -0.7071067811865475),
 6: (-1.8369701987210297e-16, -1.0),
 7: (0.7071067811865475, -0.7071067811865477)}

In [25]:

mySkeleton, conns = genBokeh_Skeleton()
mySkeleton

,coord2d,color,name
0,"(639, 123)",#7570b3,head
1,"(617, 191)",#7570b3,neck
2,"(572, 191)",#d95f02,right_shoulder
3,"(527, 236)",#d95f02,right_elbow
4,"(527, 191)",#d95f02,right_wrist
5,"(662, 191)",#1b9e77,left_shoulder
6,"(729, 258)",#1b9e77,left_elbow
7,"(729, 191)",#1b9e77,left_wrist
8,"(595, 348)",#66a61e,right_hip
9,"(595, 461)",#66a61e,right_knee


In [51]:

##### Opening joints data ######


(617, 168)

In [60]:

f = []
for (dirpath, dirnames, filenames) in os.walk("./videos"):
    f.extend(filenames)
    break
filenames

['mov1out.mp4',
 'mov3out.mp4',
 'mov1.csv',
 'mov1.mp4',
 'mov2.csv',
 'mov3.mp4',
 'mov2.mp4',
 'mov3.csv',
 'mov2out.mp4',
 'mov5.mp4',
 'mov4.csv',
 'mov4.mp4',
 'mov4out.mp4']

In [167]:

#import glob

studies=glob.glob("./videos/*.csv")
for st in range(len(studies)):
    studies[st]=studies[st].split("/")[-1].split(".")[0]
print(studies)

study_csv="./videos/"+studies[0]+".csv"

videoCoords=pd.read_csv(study_csv,sep=";",index_col=0)  
for col in mimo.columns:
    videoCoords[col]=videoCoords[col].apply(cleanStrCoords)
videoCoords.iloc[0]["head"]

['mov1', 'mov2', 'mov3', 'mov4']


(550, 191)

In [64]:
videoCoords[0:5]["neck"]

0    (550, 258)
1    (527, 236)
2    (505, 236)
3    (505, 236)
4    (504, 214)
Name: neck, dtype: object

In [91]:
#position changes
retX, retY = [],[]
lastX, lastY = (0,0)
for i in range(videoCoords.shape[0]):
    retX.append(lastX-videoCoords.iloc[i]["neck"][0])
    retY.append(lastY-videoCoords.iloc[i]["neck"][1])
    lastX=videoCoords.iloc[i]["neck"][0]
    lastY=videoCoords.iloc[i]["neck"][1]
retX[0], retY[0]=(0,0)

#print(retX, retY)

#speed


[0, 23, 22, 0, 1, 22, 22, 22, 23, 0, 22, 22, 1, 22, 1, 0, 0, 22, 0, 0, 0, 0, 0, 22, 0, 0, 0, 0, 0, -22, 0, 0, -22, 0, -1, -22, 0, -22, -22, 0, -1, -22, 0, -1, -21, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 22, 0, 1, 0, 0, 22, 0, 22, 1, 22, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -22, -1, 0, 0, -22, -22, -23, 0, -22, -1, -44, -22, -23, -22, 0, -23, 0, -23, -22, -23, 0, -22, -22, -23, -22, -1, 0, -22, -22, -22, -1, -22, 0, -23, -22, -23, -22, -22, 0, -22, -1, -22, 0, 0, 0, -1, 0, 0, -21, -1, 0, 0, 0, 0, 0, 22, 0, 1, 0, 22, 1, 22, 0, 22, 1, 22, 22, 0, 0, 22, 0, 0, 1, 0, 0, 0, 0, 0, 0, 21, 0, -21, 0, 0, -1, 0, 0, -22, 0, 0, -22, -1, 0, -22, 0, -22, -1, -21, 0, 0, 0, 0, 21, 0, 0, 0, 0, 0, 1, 0, 0, 0, 22, 0, 22, 1, 0, 22, 22, 23, 22, 22, 0, 23, 23, 22, 22, 23, 0, 22, 1, 23, 22, 22, 0, 22, 22, 1, 22, 23, 0, 22, 0, 23, 22, 23, 0, 22, 23, 22, 0, 22, 0, 1, 22, 0, 0, 0, 0, 22, 0, 0, 0, 1, 0, 0, -1, 0, 0, 0, 0, -22, -22, -1, -22, 0, 0, -22, -1, -22, -22, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 

In [166]:

p = figure(plot_width=400, plot_height=400)

# add a line renderer
p.line(np.arange(295), retX, line_width=2,color='pink')
p.line(np.arange(285), dX, line_width=2)

show(p)

In [164]:
from scipy.misc import derivative

def extCx(x):
    return videoCoords.iloc[int(x)]["neck"][0]

def f(x):
    return x**3 + x**2
dX=[]
for i in range(285):
    dX.append(derivative(extCx, i, dx=10))

def extCy(x):
    return videoCoords.iloc[int(x)]["neck"][1]

dY=[]
for i in range(285):
    dY.append(derivative(extCy, i, dx=10))


In [163]:

p = figure(plot_width=400, plot_height=400)

# add a line renderer
p.line(np.arange(295), retY, line_width=2,color='pink')
p.line(np.arange(285), dY, line_width=2)

show(p)